In [17]:
import numpy as np
from scipy.stats import truncnorm

def sigmoid(x):
    return 1 / (1 + np.e ** -x)
activation_function = sigmoid

def truncated_normal(mean=0, sd=1, low=0, upp=10):
    return truncnorm(
        (low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)


class NeuralNetwork:
            
    def __init__(self, no_of_in_nodes, no_of_out_nodes, no_of_hidden_nodes, learning_rate, bias=None):
        self.no_of_in_nodes = no_of_in_nodes
        self.no_of_hidden_nodes = no_of_hidden_nodes
        self.no_of_out_nodes = no_of_out_nodes
        self.learning_rate = learning_rate 
        self.bias = bias
        self.create_weight_matrices()
    
        
    def create_weight_matrices(self):
        bias_node = 1 if self.bias else 0 
        rad = 1 / np.sqrt(self.no_of_in_nodes + bias_node)
        X = truncated_normal(mean=0, sd=1, low=-rad, upp=rad)
        self.weights_in_hidden = X.rvs((self.no_of_hidden_nodes, 
                                        self.no_of_in_nodes + bias_node))
        rad = 1 / np.sqrt(self.no_of_hidden_nodes + bias_node)
        X = truncated_normal(mean=0, sd=1, low=-rad, upp=rad)
        self.weights_hidden_out = X.rvs((self.no_of_out_nodes, 
                                         self.no_of_hidden_nodes + bias_node))

        
    def train(self, input_vector, target_vector):
        # verification de forme
        input_vector = np.array(input_vector)
        input_vector = input_vector.reshape(input_vector.size, 1)    
        target_vector = np.array(target_vector)
        target_vector = target_vector.reshape(target_vector.size, 1)

        # Neurones d'entree
        if self.bias:
            # Ajout d'une coordonnee pour le biais
            input_vector = np.concatenate( (input_vector, [[self.bias]]) )

        # Neurones de la couche cachee
        output_vector_hidden = activation_function(self.weights_in_hidden @ input_vector)

        if self.bias:
            output_vector_hidden = np.concatenate( (output_vector_hidden, [[self.bias]]) )
            
        # Neurones de sortie
        output_vector_network = activation_function(self.weights_hidden_out @ output_vector_hidden)
        
        ###########################################################################################        

        # Equation 1 - Erreur de sortie        
        output_error = output_vector_network - target_vector
        delta_last = output_error * output_vector_network*(1-output_vector_network)
        
        # Equation 2 - Retropropagation de l'erreur
        w_h_o = self.weights_hidden_out
        if self.bias:
            w_h_o = w_h_o[:,:-1]

        hidden_errors = w_h_o.T @ delta_last 
        if not self.bias:
            delta_hidden = hidden_errors * (output_vector_hidden)*(1-output_vector_hidden)
        else:
            delta_hidden = hidden_errors * (output_vector_hidden[:-1,:])*(1-output_vector_hidden[:-1,:])
        
        # Equation 3 et 4 - Variation du cout par rapport aux biais
        Eq34_out_hidden = self.learning_rate*(delta_last@output_vector_hidden.T)
        Eq34_in_hidden = self.learning_rate*(delta_hidden@input_vector.T)
        
        # Mise à jour des poids (/!\ aux neurones de biais)
        # Poids en sortie de la couche cachee
        self.weights_hidden_out -= Eq34_out_hidden 
        
        # Poids en entree de la couche cachee            
        self.weights_in_hidden -= Eq34_in_hidden


           
    def run(self, x):
        # verification de forme
        x = np.array(x)
        x = x.reshape(x.size, 1)
        if self.bias:
            # ajout de la coordonnee biais
            x = np.concatenate( (x, [[1]]) )
        h = activation_function(self.weights_in_hidden @ x)
        if self.bias:
            h = np.concatenate( (h, [[1]]) )
        y = activation_function(self.weights_hidden_out @ h)
        return y
            
    def evaluate(self, data, labels):
        corrects, wrongs = 0, 0
        for i in range(len(data)):
            res = self.run(data[i])
            res_max = res.argmax()
            if res_max == labels[i]:
                corrects += 1
            else:
                wrongs += 1
        return corrects, wrongs


In [27]:
import numpy as np
from scipy.stats import truncnorm
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt

n_sample=5000
data, labels = make_blobs(n_samples=n_sample, n_features=3, random_state=0, centers=2, cluster_std=1.2)
size_of_learn_sample = int(n_sample * 0.8)
size_of_test_sample = n_sample-size_of_learn_sample
X_learn_data = data[:size_of_learn_sample]
X_test_data = data[-size_of_test_sample:]
y_learn_data = labels[:size_of_learn_sample]
y_test_data = labels[-size_of_test_sample:]

simple_network = NeuralNetwork(no_of_in_nodes=3, 
                               no_of_out_nodes=2, 
                               no_of_hidden_nodes=4,
                               learning_rate=0.1,
                               bias=1)
    
labels = (np.arange(2) == y_learn_data.reshape(y_learn_data.size, 1))
labels = labels.astype(np.float64)

n = len(X_learn_data[:,1])
epoch = 10
for e in range(epoch):
    perm = np.arange(n)
    perm = np.random.permutation(perm)
    for i in perm:
        simple_network.train(X_learn_data[i], labels[i])
    correct, wrong = simple_network.evaluate(X_learn_data, y_learn_data)
    print(wrong, correct, "->Taux de gagnage", 100*correct/(correct+wrong),"%")


    

correct, wrong = simple_network.evaluate(X_test_data, y_test_data)
print(wrong, correct, "->taux de perdage", 100*wrong/(correct+wrong),"%")


27 3973 ->Taux de gagnage 99.325 %
27 3973 ->Taux de gagnage 99.325 %
30 3970 ->Taux de gagnage 99.25 %
25 3975 ->Taux de gagnage 99.375 %
25 3975 ->Taux de gagnage 99.375 %
27 3973 ->Taux de gagnage 99.325 %
24 3976 ->Taux de gagnage 99.4 %
27 3973 ->Taux de gagnage 99.325 %
24 3976 ->Taux de gagnage 99.4 %
26 3974 ->Taux de gagnage 99.35 %
10 990 ->taux de perdage 1.0 %
